In [1]:
'''
IMPORT LIBRARIES AND PACKAGES 
'''
#libraries for data manipulation and analysis 
import pandas as pd 
import numpy as np
from collections import Counter
import math

#libraries for date and time handling 
from dateutil import parser
import datetime


#libraries for excel formatting 
from openpyxl.utils.cell import get_column_letter
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule
import xlsxwriter


#libraries for string similarity and fuzzy matching 
import jellyfish
from Levenshtein import distance as ld
from fuzzywuzzy import fuzz
import textdistance as td

#libraries with general utility functions 
import random
import os
from termcolor import cprint
from collections import _count_elements 
from collections import Counter

In [2]:
#variables for output files
file_modifier = "david_test1"
output_pathway = r'C:\Users\NguyenD29\OneDrive - Cedars-Sinai Health System\Documents\Trainings\Marco Documentation\OSCAR_name_matching\oscar_name_matching\outputs' 

#flags
remove_var = 1 
drop_pat_ids = 1

## BRING IN FILES

In [35]:
#read in files - mr: medical records, dc: death certificates 
medical_records = pd.read_excel(r"C:\Users\NguyenD29\OneDrive - Cedars-Sinai Health System\Documents\Trainings\Marco Documentation\OSCAR_name_matching\oscar_name_matching\data\ForDeathCertMatch_7_17_2025.xlsx", dtype = 'string')
death_certs = pd.read_excel(r"C:\Users\NguyenD29\OneDrive - Cedars-Sinai Health System\Documents\Trainings\Marco Documentation\OSCAR_name_matching\oscar_name_matching\data\Norby_CCDF_040125_063025.xlsx", dtype = 'string')
gender = pd.read_excel(r'C:\Users\NguyenD29\OneDrive - Cedars-Sinai Health System\Documents\Trainings\Marco Documentation\OSCAR_name_matching\oscar_name_matching\data\oscar_gender.xlsx', dtype={'strMRN' : str, 'gender' : str}) 

df = medical_records.copy()
matchto = death_certs.copy()

In [28]:
#checking for columns with same name in medical records and death certificate files 
overlap = set(medical_records.columns) & set(death_certs.columns)

#if there is an overlap
if len(overlap) > 0:
    #rename column in death_certs (we will match to it later) 
    raise Exception(f'Column overlap detected: {overlap}. Renaming in death certificates dataframe')

    for col in overlap:
        death_certs = death_certs.rename(columns = {col: col+'_matchto'})
        print(f'Renaming column from {col} to {col+"_matchto"}') 
else: 
    print('No column overlap detected')

#checking for commas in column names - need to change them in the input file 
erroneouslist_dc = list()
for col in list(death_certs.columns):
    if ',' in col:
        erroneouslist_dc.append(col)
        
erroneouslist_mr = list()
for col in list(medical_records.columns):
    if ',' in col:
        erroneouslist_mr.append(col)
    
if len(erroneouslist_dc) > 0:
    raise Exception('PLEASE FIX:\nThere are commas (,) found in the following death_certs columns: {}\nThese will cause problems later down the line. Please manually change them in the input file.'.format(erroneouslist_dc))
if len(erroneouslist_mr) > 0:
    raise Exception('PLEASE FIX:\nThere are commas (,) found in the following medical_records columns: {}\nThese will cause problems later down the line. Please manually change them in the input file.'.format(erroneouslist_mr))


No column overlap detected


In [29]:
#split name into first, middle, last for medical records dataframe 
if 'PAT_NAME' in medical_records:
    medical_records['First Name'] = medical_records['PAT_NAME'].apply(lambda x : x.split(',')[1].split(' ')[0]) 
    medical_records['Middle Name'] = medical_records['PAT_NAME'].apply(lambda x : x.split(',')[1].split(' ')[1] if ' ' in x.split(',')[1] else None) 
    medical_records['Last Name'] = medical_records['PAT_NAME'].apply(lambda x : x.split(',')[0])

In [68]:
medical_records = df.copy()
death_certs = matchto.copy()

## CLEAN SOCIAL SECURITY NUMBER

In [69]:
def ssn_cleaner(x, invalid_ssns):
    '''
    function to process ssn to filter for ssns that would be considered invalid 
    - more than six 0's
    - more than seven 1's
    - more than six 9's
    - all 0's and 1's 
    '''
    #replace trailing 0 
    x = x.replace('.0', '')

    #return nothing if ssn is in invalid ssns list 
    if x in invalid_ssns:
        return None

    #add leading 0's
    x = '0'*(9-len(x)) + str(x)

    #find counts of each digit and return none for invalid ssns 
    digit_counts = Counter(x)
    if digit_counts['0'] > 6 or digit_counts['1'] > 7 or digit_counts['9'] > 6 or digit_counts.keys() == {0,1}: 
        return None
    else:
        return x
        

#clean up ssn column in medical records dataframe
if 'SSN' in medical_records.columns:
    #clean up dashes
    medical_records['SSN'] = medical_records['SSN'].apply(lambda x: str(x).replace('-', ''))

    #get invalid ssns - threshold is if there is 4 or more of the same one
    counts = medical_records['SSN'].value_counts().to_frame().reset_index()
    invalid_ssn = counts[counts['count'] > 4]['SSN'].to_list()
    invalid_ssn += ['<NA>', 'nan', None, '123456789', '0', 0]

    #use helper function to clean ssn 
    medical_records['SSN'] = medical_records['SSN'].apply(lambda x: ssn_cleaner(x, invalid_ssn))


#clean up ssn in death certificates dataframe
if 'F31' in death_certs.columns: 
    #clean up dashes
    death_certs['F31'] = death_certs['F31'].apply(lambda x: str(x).replace('-', ''))

    #get invalid ssns - threshold is if there is 4 or more of the same one
    counts = death_certs['F31'].value_counts().to_frame().reset_index()
    invalid_ssn = counts[counts['count'] > 4]['F31'].to_list()
    invalid_ssn += ['<NA>', 'nan', None, '123456789', '0', 0]

    #use helper function to clean ssn 
    death_certs['F31'] = death_certs['F31'].apply(lambda x: ssn_cleaner(x, invalid_ssn))


In [82]:
################# COLUMN ASSOCIATION CHUNK

### NORMAL RUN!
dfdob = 'BIRTH_DATE' #make this a tkinter menu later - modify code from FullSuiteGeolocation header - picking columns from drop down list
dfdod = 'DEATH_DATE'
dffn = 'First Name'
dfmn = 'Middle Name'
dfln = 'Last Name'
dfunids = ['PAT_ID']
dfaddress = 'ADD_LINE_1'
dfsex = 'GENDER'
dfarrestloc_a = 'BlankCol_df1'
dfarrestloc_b = 'BlankCol_df2'

## ONE OFF AUTOPSY RUN:
# dfdob = 'Demographics.DATE OF BIRTH'
# dfdod = 'Date of death'
# dffn = 'First Name'
# dfmn = 'Middle Name'
# dfln = 'Last Name'
# dfunids = ['SUDS #']
# dfaddress = 'BlankCol_df'
# dfsex = 'BlankCol_df'
# dfarrestloc_a = 'BlankCol_df'
# dfarrestloc_b = 'BlankCol_df'

# ## ORIGINAL RUN
# matchtodob = 'DOB'
# matchtodod = 'DOD'
# matchtofn = 'FNAME'
# matchtomn = 'MNAME'
# matchtoln = 'LNAME' #see above
# matchtotarget = 'Snumber'
# matchtounids = ['Snumber']
# matchtoaddress = 'ADDR1'
# matchtoaddress2 = 'ADDR2' #
# matchtocity = 'CITY_matchto'	 #
# matchtosex = 'SEX' #

## DC RUN
# matchtodob = 'dob'
# matchtodod = 'dod'
# matchtofn = 'First_name'
# matchtomn = 'middle_name'
# matchtoln = 'last_name' #see above
# matchtotarget = 'SFN'
# matchtounids = ['SFN']
# matchtoaddress = 'home addr1'
# matchtoaddress2 = 'home_addre' #
# matchtocity = 'home_city'	 #

## New Combined CCDF Norby run
matchtodob = 'F8'
matchtodod = 'F20'
matchtofn = 'F3'
matchtomn = 'F4'
matchtoln = 'F5' #see above
matchtotarget = 'F1'
matchtounids = ['F1']
matchtoaddress = 'F55'
matchtoaddress2 = 'F56' #
matchtocity = 'F57'	 #
matchtosex = 'F19' #have to get this column from "C:\Users\MathiasM\Box\OSCAR Death data\Norby_CCDF_010123_063024_combined.xlsx"
matchtodoa = 'BlankCol_matchto1'
matchtossn = 'F31'
matchtoarrestloc_a = 'F132'
matchtoarrestloc_b = 'F133'
matchtoaddress_arrest_city = 'F134'

##One-off autopsy run:
# matchtodob = 'Identification - Date of birth'
# matchtodod = "Identification - Date of death"
# matchtofn = 'Identification - First name'
# matchtomn = 'BlankCol_matchto'
# matchtoln = 'Identification - Last name'
# matchtotarget = "Identification - Case number"
# matchtounids = ["Identification - Case number"]
# matchtoaddress = 'BlankCol_matchto'
# matchtoaddress2 = 'BlankCol_matchto'
# matchtocity = 'BlankCol_matchto'
# matchtosex = 'BlankCol_matchto'
# matchtodoa = 'BlankCol_matchto'
#matchtossn 

# matchtoarrestloc_a = 'BlankCol_matchto'
# matchtoarrestloc_b = 'BlankCol_matchto'
# matchtoaddress_arrest_city = 'BlankCol_matchto'

In [ ]:
################# COLUMN ASSOCIATION CHUNK

### NORMAL RUN!
medical_record_dob = 'BIRTH_DATE' #make this a tkinter menu later - modify code from FullSuiteGeolocation header - picking columns from drop down list
medical_record_dod = 'DEATH_DATE'
medical_record_fn = 'First Name'
medical_record_mn = 'Middle Name'
medical_record_ln = 'Last Name'
medical_record_unids = ['PAT_ID']
medical_record_address = 'ADD_LINE_1'
medical_record_sex = 'GENDER'
medical_record_arrestloc_a = 'BlankCol_medical_record1'
medical_record_arrestloc_b = 'BlankCol_medical_record2'

## ONE OFF AUTOPSY RUN:
# medical_record_dob = 'Demographics.DATE OF BIRTH'
# medical_record_dod = 'Date of death'
# medical_record_fn = 'First Name'
# medical_record_mn = 'Middle Name'
# medical_record_ln = 'Last Name'
# medical_record_unids = ['SUDS #']
# medical_record_address = 'BlankCol_medical_record'
# medical_record_sex = 'BlankCol_medical_record'
# medical_record_arrestloc_a = 'BlankCol_medical_record'
# medical_record_arrestloc_b = 'BlankCol_medical_record'

# ## ORIGINAL RUN
# death_certs_dob = 'DOB'
# death_certs_dod = 'DOD'
# death_certs_fn = 'FNAME'
# death_certs_mn = 'MNAME'
# death_certs_ln = 'LNAME' #see above
# death_certs_target = 'Snumber'
# death_certs_unids = ['Snumber']
# death_certs_address = 'ADDR1'
# death_certs_address2 = 'ADDR2' #
# death_certs_city = 'CITY_death_certs_'	 #
# death_certs_sex = 'SEX' #

## DC RUN
# death_certs_dob = 'dob'
# death_certs_dod = 'dod'
# death_certs_fn = 'First_name'
# death_certs_mn = 'middle_name'
# death_certs_ln = 'last_name' #see above
# death_certs_target = 'SFN'
# death_certs_unids = ['SFN']
# death_certs_address = 'home addr1'
# death_certs_address2 = 'home_addre' #
# death_certs_city = 'home_city'	 #

## New Combined CCDF Norby run
death_certs_dob = 'F8'
death_certs_dod = 'F20'
death_certs_fn = 'F3'
death_certs_mn = 'F4'
death_certs_ln = 'F5' #see above
death_certs_target = 'F1'
death_certs_unids = ['F1']
death_certs_address = 'F55'
death_certs_address2 = 'F56' #
death_certs_city = 'F57'	 #
death_certs_sex = 'F19' #have to get this column from "C:\Users\MathiasM\Box\OSCAR Death data\Norby_CCDF_010123_063024_combined.xlsx"
death_certs_doa = 'BlankCol_death_certs1'
death_certs_ssn = 'F31'
death_certs_arrestloc_a = 'F132'
death_certs_arrestloc_b = 'F133'
death_certs_address_arrest_city = 'F134'

##One-off autopsy run:
# death_certs_dob = 'Identification - Date of birth'
# death_certs_dod = "Identification - Date of death"
# death_certs_fn = 'Identification - First name'
# death_certs_mn = 'BlankCol_death_certs'
# death_certs_ln = 'Identification - Last name'
# death_certs_target = "Identification - Case number"
# death_certs_unids = ["Identification - Case number"]
# death_certs_address = 'BlankCol_death_certs'
# death_certs_address2 = 'BlankCol_death_certs'
# death_certs_city = 'BlankCol_death_certs'
# death_certs_sex = 'BlankCol_death_certs'
# death_certs_doa = 'BlankCol_death_certs'
# death_certs_ssn 

# death_certs_arrestloc_a = 'BlankCol_death_certs'
# death_certs_arrestloc_b = 'BlankCol_death_certs'
# death_certs_address_arrest_city = 'BlankCol_death_certs'

## QUESTION
- do we need to keep this naming section?
- we end up deleting themall too
- also why does it only delete the last var? 


In [81]:
bc_matchto = []
bc_df = []
count_m = 1

for var in list(locals()):
    if 'matchto' in var:
        if type(eval(var)) == str:
           if 'blankcol' in eval(var).lower() and '_first' not in eval(var).lower():
                print('Renamed {} from {} to {}'.format(var, eval(var), eval(var)+str(count_m)))

                exec("{}='{}{}'".format(var, eval(var), count_m))

                count_m = count_m + 1
                bc_matchto.append(eval(var)) #bc var already changed, don't need to append count_m
                if eval(var) not in matchto.columns:
                    matchto.insert(matchto.shape[1], eval(var), None)
    if 'df' in var:
        if type(eval(var)) == str:
            if 'blankcol' in eval(var).lower() and '_first' not in eval(var).lower():
                print('Renamed {} from {} to {}'.format(var, eval(var), eval(var)+str(count_m)))

                exec("{}='{}{}'".format(var, eval(var), count_m))
                
                count_m = count_m + 1
                bc_df.append(eval(var)) #bc var already changed, don't need to append count_m
                if eval(var) not in df.columns:
                    df.insert(df.shape[1], eval(var), None)

del bc_matchto, bc_df, count_m, var

Renamed dfarrestloc_a from BlankCol_df1 to BlankCol_df11
Renamed dfarrestloc_b from BlankCol_df2 to BlankCol_df22
Renamed matchtodoa from BlankCol_matchto1 to BlankCol_matchto13


In [ ]:



# if not 'BlankCol_matchto1' in matchto.columns:
#     matchto.insert(matchto.shape[1], 'BlankCol_matchto1', None) #changed default value from '' to None, could cause errors. Alternative is to update subfuncs s.t they account
#     #for blank strings. Notably date_diffs_noabs
# if not 'BlankCol_matchto2' in matchto.columns:
#     matchto.insert(matchto.shape[1], 'BlankCol_matchto2', None)
# if not 'BlankCol_df1' in df.columns:
#     df.insert(df.shape[1], 'BlankCol_df1', None)
# if not 'BlankCol_df2' in df.columns:
#     df.insert(df.shape[1], 'BlankCol_df2', None)

def gendertypeswitch(x): #where x is a single entry
    if not pd.isna(x):
        if x == 'F':
            return '1'
        elif x == 'M':
            return '2'
        elif x == '1' or x == 1: #protecting other type if present already (usually for df)
            return '1' 
        elif x == '2' or x == 2: #protecting other type if present already (usually for df)
            return '2'
        else:
            return None
    else:
        return None
preclean_gender_matchto = list(matchto[matchtosex].unique())
preclean_gender_df = list(df[dfsex].unique())
#matchto[matchtosex] = matchto[matchtosex].apply(lambda x : '1' if x == 'F' else '2') #priya said F = 1, M = 2. THIS IS ONLY FOR THIS CHUNK! COMMENT OUT IF NOT USING!
matchto[matchtosex] = matchto[matchtosex].apply(lambda x : gendertypeswitch(x))
df[dfsex] = df[dfsex].apply(lambda x : gendertypeswitch(x))



#Created here:
matchtofn2 = matchtofn+'_f2'
matchtoln2 = matchtoln+'_f2'

matchtofn3 = matchtofn+'_f3'
matchtoln3 = matchtoln+'_f3'

matchtofn1 = matchtofn+'_f1'
matchtoln1 = matchtoln+'_f1'
matchtoaddress_1 = matchtoaddress+"_first"

######Created here, no need to modify:
dffn2 = dffn+'_f2'
dfln2 = dfln+'_f2'

dffn3 = dffn+'_f3'
dfln3 = dfln+'_f3'

dffn1 = dffn+'_f1'
dfln1 = dfln+'_f1'

dfaddress_1 = dfaddress+'_first'

# if 'matchtossn' in locals(): #make this more modular...
#     matchto_columns = [matchtodob, matchtodod, matchtofn, matchtomn, matchtoln, matchtotarget, 
#                     matchtofn2, matchtoln2, matchtofn3, matchtoln3, matchtofn1, matchtoln1, matchtossn, matchtoaddress_1, matchtoaddress,
#                     matchtoaddress2, matchtocity, matchtosex, matchtoarrestloc_a, matchtoarrestloc_b, matchtoaddress_arrest_city] #+ [i for i in matchto.columns if 'blankcol']
# else: 
#     matchto_columns = [matchtodob, matchtodod, matchtofn, matchtomn, matchtoln, matchtotarget, 
#                     matchtofn2, matchtoln2, matchtofn3, matchtoln3, matchtofn1, matchtoln1, matchtoaddress_1, matchtoaddress,
#                     matchtoaddress2, matchtocity, matchtosex, matchtoarrestloc_a, matchtoarrestloc_b, matchtoaddress_arrest_city]

matchto_columns = []
for var in list(locals()):
    if str(var[0:7]) == 'matchto' and type(eval(var)) == str:
        #print(var, var[0:7])
        matchto_columns.append(eval(var))

def date_parsing(x):
    if not pd.isna(x):
        try:
            if type(x) != datetime.date:
                return parser.parse(str(x)).date()
            else:
                return x
        except Exception as e:
            #print('Could not date parse: {} due to {}'.format(x, e))
            return None
    else:
        return None

def DTfix(subset_string, dataframe, list_of_date_columns):
    for z in list_of_date_columns:
        if z in dataframe.columns:
            print('{} {} dtype'.format(subset_string, z), type(dataframe[z].iloc[0]))
            if type(dataframe[z].iloc[0]) != datetime.date:
                print('Converting {} {} dtype...'.format(subset_string, z))
                dataframe[z] = dataframe[z].apply(lambda x : date_parsing(x))
                print('Done converting... checking...')
                date_test = dataframe[~pd.isna(dataframe[z])][z].iloc[round(len(dataframe[~pd.isna(dataframe[z])])/2)]
                print('     ',date_test, type(date_test),'\n')
        else:
            print('Could not find {} in {}\n'.format(z, subset_string))

DTfix('df', df, [dfdob, dfdod, 'MAX_ENC_DATE'])
DTfix('matchto', matchto, [matchtodob, matchtodod])

print('\nCleaning name columns...')

def lower_spacerm_sub(x):
    if pd.isna(x):
        return ''
    
    if type(x) == str:
        return str(x).lower().replace(' ', '').replace('-','').replace(',', '').replace("'", '').replace('.', '')
   
def lower_spacerm(dataframe, list_of_name_columns):
    for z in list_of_name_columns:
        dataframe[z] = dataframe[z].apply(lambda x : lower_spacerm_sub(x))

lower_spacerm(df, [dffn, dfmn, dfln])
lower_spacerm(matchto, [matchtofn, matchtomn, matchtoln])


print('Forming "first_*" columns') #occurs after lower_spacerm so don't have to reapply all that stuff!
#functionalize this!!!
def nameshorten(x, number_of_characters):
    if not pd.isna(x):
        return x[0:number_of_characters]
    else:
        return None
df[dffn2] = df[dffn].apply(lambda x : nameshorten(x,2))
df[dfln2] = df[dfln].apply(lambda x : nameshorten(x,2))
matchto[matchtofn2] = matchto[matchtofn].apply(lambda x : nameshorten(x,2))
matchto[matchtoln2] = matchto[matchtoln].apply(lambda x : nameshorten(x,2))

df[dffn3] = df[dffn].apply(lambda x : nameshorten(x,3))
df[dfln3] = df[dfln].apply(lambda x : nameshorten(x,3))
df[dffn1] = df[dffn].apply(lambda x : nameshorten(x,1))
df[dfln1] = df[dfln].apply(lambda x : nameshorten(x,1))

matchto[matchtofn3] = matchto[matchtofn].apply(lambda x : nameshorten(x,3))
matchto[matchtoln3] = matchto[matchtoln].apply(lambda x : nameshorten(x,3))
matchto[matchtofn1] = matchto[matchtofn].apply(lambda x : nameshorten(x,1))
matchto[matchtoln1] = matchto[matchtoln].apply(lambda x : nameshorten(x,1))


#Forming address_1
print('Forming "address_1" columns')

def address_split(x):
    x=str(x)
    if not pd.isna(x) and x != 'nan' and x != '<NA>':
        return x.split(' ')[0]
    else:
        return None

df[dfaddress_1] = df[dfaddress].apply(lambda x : address_split(x))    
matchto[matchtoaddress_1] = matchto[matchtoaddress].apply(lambda x : address_split(x))  

#Also dropping duplicate rows in both:
def dupedrop(dataframe, subset_string, list_of_unid_columns):
    prel = len(dataframe)
    temp = dataframe.drop_duplicates()
    postl = len(temp)
    print('Dropped {} duplicates from {}'.format(prel-postl, subset_string))

    for idcol in list_of_unid_columns:
        f1list=dataframe[idcol].value_counts().to_frame()
        #f2list=dataframe['F2'].value_counts().to_frame()
        f1list = f1list[f1list['count'] == 2]
        #f2list = f2list[f2list['count'] == 2]
        print(len(f1list))
        if len(f1list) > 0:
            print('There is a duplicate in the {} column - see the following records:\n {}'.format(idcol, f1list))

    return temp

print('\n')
df = dupedrop(df, 'df', dfunids)
matchto = dupedrop(matchto, 'matchto', matchtounids)

df_columns = list(df.columns) + ['nametag_df'] #locking columns here so we can filter using this later!
print(df_columns)